##Notebook overview:
- In this notebook we load pictures which should be predicted and move those pictures into the determined folder based on the prediction made by the model.
- First we have to install TFOD API (as always)
- Afterwards we create a list of image_paths 
- Then we load the model
- Finally we make predictions and collect the bounding box data and parse it into the dataframe for each detection
- Save the dataframe in csv format 

In [ ]:
#connect to drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## **Cloning TFOD 2.0 Github** needed for inferencing

In [ ]:
!git clone https://github.com/tensorflow/models.git

Cloning into 'models'...
remote: Enumerating objects: 69183, done.
remote: Total 69183 (delta 0), reused 0 (delta 0), pack-reused 69183
Receiving objects: 100% (69183/69183), 577.29 MiB | 31.35 MiB/s, done.
Resolving deltas: 100% (48740/48740), done.


In [ ]:
cd /content/models/research

/content/models/research


In [ ]:
!protoc object_detection/protos/*.proto --python_out=.

In [ ]:
!git clone https://github.com/cocodataset/cocoapi.git

Cloning into 'cocoapi'...
remote: Enumerating objects: 975, done.
remote: Total 975 (delta 0), reused 0 (delta 0), pack-reused 975
Receiving objects: 100% (975/975), 11.72 MiB | 10.61 MiB/s, done.
Resolving deltas: 100% (576/576), done.


In [ ]:
cd cocoapi/PythonAPI

/content/models/research/cocoapi/PythonAPI


In [ ]:
!make

python setup.py build_ext --inplace
running build_ext
cythoning pycocotools/_mask.pyx to pycocotools/_mask.c
/usr/local/lib/python3.7/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /content/models/research/cocoapi/PythonAPI/pycocotools/_mask.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
building 'pycocotools._mask' extension
creating build
creating build/common
creating build/temp.linux-x86_64-3.7
creating build/temp.linux-x86_64-3.7/pycocotools
x86_64-linux-gnu-gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fdebug-prefix-map=/build/python3.7-pX47U3/python3.7-3.7.12=. -fstack-protector-strong -Wformat -Werror=format-security -g -fdebug-prefix-map=/build/python3.7-pX47U3/python3.7-3.7.12=. -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/local/lib/python3.7/dist-packages/n

### Install the Object Detection API

In [ ]:
cd /content/models/research

/content/models/research


In [ ]:
!cp object_detection/packages/tf2/setup.py .
!python -m pip install .

Processing /content/models/research
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
     |████████████████████████████████| 10.1 MB 8.4 MB/s 
     |████████████████████████████████| 352 kB 55.9 MB/s 
     |████████████████████████████████| 2.2 MB 44.6 MB/s 
     |████████████████████████████████| 23.4 MB 1.2 MB/s 
     |████████████████████████████████| 234 kB 70.5 MB/s 
     |████████████████████████████████| 636 kB 59.4 MB/s 
     |████████████████████████████████| 99 kB 8.8 MB/s 
     |████████████████████████████████| 43 kB 1.6 MB/s 
     |████████████████████████████████| 4.9 MB 39.1 MB/s 
     |████████████████████████████████| 1.1 MB 44.

In [ ]:
# From within TensorFlow/models/research/
#test if tensorflow downloaded successfully
!python object_detection/builders/model_builder_tf2_test.py

2022-02-20 22:36:31.195222: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Running tests under Python 3.7.12: /usr/bin/python3
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_deepmac
W0220 22:36:32.558691 140685205174144 model_builder.py:1100] Building experimental DeepMAC meta-arch. Some features may be omitted.
INFO:tensorflow:time(__main__.ModelBuilderTF2Test.test_create_center_net_deepmac): 2.13s
I0220 22:36:33.327841 140685205174144 test_util.py:2374] time(__main__.ModelBuilderTF2Test.test_create_center_net_deepmac): 2.13s
[       OK ] ModelBuilderTF2Test.test_create_center_net_deepmac
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_model0 (customize_head_params=True)
INFO:tensorflow:time(__main__.ModelBuilderTF2Test.test_create_center_net_model0 (customize_head_params=True)): 1.23s
I0220 22:36:34.557748 140685205174144 test_util.py:2374] time(__main__.ModelBuilderTF2Test.test_cr

## Create a list of image_paths

In [ ]:
import os

In [ ]:
#create a list of pcitures filepaths
image_dir= '/content/drive/MyDrive/training_demo/images/test_below50' 
files = os.listdir(image_dir)
#print(files)
image_list = []
for file in files:
  if file.endswith("jpg"):
    image_list.append(image_dir +"/" + file)
print(image_list)

['/content/drive/MyDrive/training_demo/images/test_below50/image_tcom14532bewlow_50_Tchibologo_valscore_0.33926126.jpg', '/content/drive/MyDrive/training_demo/images/test_below50/image_tcom15199bewlow_50_Tchibologo_valscore_0.4389768.jpg', '/content/drive/MyDrive/training_demo/images/test_below50/image_tcom14414bewlow_50_Tchibologo_valscore_0.3488722.jpg', '/content/drive/MyDrive/training_demo/images/test_below50/image_tcom14748bewlow_50_Tchibologo_valscore_0.19060473.jpg', '/content/drive/MyDrive/training_demo/images/test_below50/image_tcom14723bewlow_50_Tchibologo_valscore_0.32619238.jpg', '/content/drive/MyDrive/training_demo/images/test_below50/image_tcom14458bewlow_50_Tchibologo_valscore_0.2688396.jpg', '/content/drive/MyDrive/training_demo/images/test_below50/image_tcom14118bewlow_50_Tchibologo_valscore_0.2202396.jpg', '/content/drive/MyDrive/training_demo/images/test_below50/image_tcom13917bewlow_50_Tchibologo_valscore_0.22463761.jpg', '/content/drive/MyDrive/training_demo/image

##Load the model

In [ ]:
import tensorflow as tf

In [ ]:
# load the model

import time
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils

path_to_saved_model = "/content/drive/MyDrive/training_demo/new_exported_models/Exported_centernet_512/saved_model"

print('Loading model...', end='')
start_time = time.time()

# load saved model 
detect_fn = tf.saved_model.load(path_to_saved_model)

end_time = time.time()
elapsed_time = end_time - start_time
print('Done! Took {} seconds'.format(elapsed_time))

## Predict pictures and move them into directory based on the prediction

If there are problems with cv2 ("registerMatetype" or something like that) run the cell below and restart runtime

In [ ]:
!pip uninstall opencv-python
!pip install opencv-python==4.5.2.52
!pip uninstall opencv-contrib-python
!pip install opencv-contrib-python==4.5.2.52
!pip uninstall opencv-python-headless
!pip install opencv-python-headless==4.5.2.52

#Create Empty Dataframe

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.DataFrame()

In [ ]:
df["image_id"]=np.nan
df["width"]=np.nan
df["height"]= np.nan
df["class"]=np.nan
df["xmin"]=np.nan
df["ymin"]=np.nan
df["xmax"]=np.nan
df["ymax" ] = np.nan

In [ ]:
df.head()

,image_id,width,height,class,xmin,ymin,xmax,ymax


### Predict the Pictures and write annotation information to csv file

In [ ]:
from tensorflow.core import framework
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'    # Suppress TensorFlow logging (1)
import pathlib
import tensorflow as tf
import cv2
import argparse

# provide path to label_map
path_to_labels = '/content/drive/MyDrive/training_demo/annotations/label_map.pbtxt'

# load label map
category_index = label_map_util.create_category_index_from_labelmap(path_to_labels,
                                                                    use_display_name=True)
import numpy as np 
import warnings


i=0
for image_path in image_list:
 #assign index with enumerate didnt't work
  i +=1
  image = cv2.imread(image_path)
  #some pictures can not be grabed by cv2.cvtColor. Haven't figured out why ao I handled it with a try/except.
  try:
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  except:
    continue 
  image_expanded = np.expand_dims(image_rgb, axis=0)

  # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
  input_tensor = tf.convert_to_tensor(image)
  # The model expects a batch of images, so add an axis with `tf.newaxis`.
  input_tensor = input_tensor[tf.newaxis, ...]

  # input_tensor = np.expand_dims(image_np, 0)
  detections = detect_fn(input_tensor)
  
  # All outputs are batches tensors.
  # Convert to numpy arrays, and take index [0] to remove the batch dimension.
  # We're only interested in the first num_detections.
  num_detections = int(detections.pop('num_detections'))
  detections = {key: value[0, :num_detections].numpy()
                for key, value in detections.items()}
  detections['num_detections'] = num_detections

  # detection_classes should be ints.
  detections['detection_classes'] = detections['detection_classes'].astype(np.int64)
  
  #grab bounding box coordinates and write them to dataframe 

  d=0
  for x in detections['detection_scores']:

    if detections['detection_scores'][d] > 0.5:
      base=os.path.basename(image_path)
      image_name= base
      annotations= {'image_id': image_name, "width": image.shape[0] , "height": image.shape[1] , 'class': detections["detection_classes"][d] , 'xmin': detections['detection_boxes'][d][0] * 1000 , 'ymin': detections['detection_boxes'][d][1] *1000, 'xmax': detections['detection_boxes'][d][2] *1000  , 'ymax': detections['detection_boxes'][d][3] *1000}
      df = df.append(annotations, ignore_index = True)
      d+=1
      print("appended")
    else:
      break



#Show first rows of dataframe and save it as csv

In [ ]:
df.head(12)

,image_id,width,height,class,xmin,ymin,xmax,ymax
0,image_tcom14532bewlow_50_Tchibologo_valscore_0...,1600.0,1200.0,1.0,646.007121,14.951380,721.039355,57.586543
1,image_tcom15199bewlow_50_Tchibologo_valscore_0...,1600.0,1200.0,1.0,377.981275,211.485490,486.922413,341.521084
2,image_tcom14458bewlow_50_Tchibologo_valscore_0...,973.0,1600.0,1.0,519.294322,124.019906,561.302841,159.849480
3,image_tcom14036bewlow_50_Tchibologo_valscore_0...,1600.0,1200.0,1.0,702.228189,540.342689,788.585544,671.339273
4,image_tcom13929bewlow_50_Tchibologo_valscore_0...,1600.0,900.0,1.0,493.547350,653.915644,524.362624,694.548488
5,image_tcom13669bewlow_50_Tchibologo_valscore_0...,1600.0,1200.0,1.0,518.450201,755.100131,578.834593,816.668868
6,image_tcom13472bewlow_50_Tchibologo_valscore_0...,1200.0,1600.0,1.0,322.725207,371.735722,356.937438,397.498816
7,image_tcom13241bewlow_50_Tchibologo_valscore_0...,1600.0,1200.0,1.0,311.871171,275.766343,399.203181,384.814352
8,image_tcom13775bewlow_50_Tchibologo_valscore_0...,1600.0,900.0,1.0,659.649968,435.807645,687.648416,492.033660
9,image_tcom13405bewlow_50_Tchibologo_valscore_0...,1600.0,1200.0,1.0,377.590239,318.602890,399.760485,346.533686


In [ ]:
df.to_csv(r"/content/drive/MyDrive/training_demo/images/test.csv", index=False)